In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

import kraft

In [ ]:
directory_path = "/home/kwat/garden/data/cancer_cell_line/copy_number/"

kraft.path.path(directory_path)

## Make gene_x_cell_line_x_copy_number

In [ ]:
gene_x_cell_line_x_copy_number = pd.read_csv(
    "{}/CCLE_gene_cn.csv".format(directory_path), index_col=0
).T

gene_x_cell_line_x_copy_number

In [ ]:
gene_x_cell_line_x_copy_number.index = tuple(
    label.split()[0] for label in gene_x_cell_line_x_copy_number.index.to_numpy()
)

gene_x_cell_line_x_copy_number.index.name = "Gene"

In [ ]:
gene_x_cell_line_x_copy_number.columns = kraft.cell_line.rename(
    gene_x_cell_line_x_copy_number.columns.to_numpy()
)

In [ ]:
kraft.dataframe.error_axes(gene_x_cell_line_x_copy_number)

gene_x_cell_line_x_copy_number.to_csv(
    "{}/gene_x_cell_line_x_copy_number.tsv".format(directory_path), sep="\t",
)

gene_x_cell_line_x_copy_number

In [ ]:
mean = np.nanmean(gene_x_cell_line_x_copy_number.to_numpy())

std = np.nanstd(gene_x_cell_line_x_copy_number.to_numpy())

loss = mean - std * 3

gain = mean + std * 3

loss, gain

In [ ]:
def split_gene_loss_gain(copy_numbers, loss, gain):

    gene_loss_gain_x_label = np.full((2, copy_numbers.size), 0)

    for i, copy_number in enumerate(copy_numbers.to_numpy()):

        if copy_number < loss:

            gene_loss_gain_x_label[0, i] = 1

        elif gain < copy_number:

            gene_loss_gain_x_label[1, i] = 1

    axis_0_has_1 = (gene_loss_gain_x_label == 1).any(axis=1)

    format_ = "{} {{}}".format(copy_numbers.name)

    return pd.DataFrame(
        gene_loss_gain_x_label[axis_0_has_1],
        index=np.asarray((format_.format("loss"), format_.format("gain")))[
            axis_0_has_1
        ],
        columns=copy_numbers.index,
    )

In [ ]:
file_path = "{}/gene_loss_gain_x_cell_line_x_01.tsv".format(directory_path)

n = gene_x_cell_line_x_copy_number.shape[0]

n_per_print = n // 100

with open(file_path, mode="w") as io:

    for i, (_, copy_numbers) in enumerate(gene_x_cell_line_x_copy_number.iterrows()):

        if i % n_per_print == 0:

            print("{}/{} ...".format(i + 1, n))

        split_gene_loss_gain(copy_numbers, loss, gain).to_csv(
            io, sep="\t", header=i == 0, mode="a"
        )

gene_loss_gain_x_cell_line = pd.read_csv(file_path, sep="\t", index_col=0)

gene_loss_gain_x_cell_line

In [ ]:
gene_loss_gain_x_cell_line.sum(axis=1).sort_values()